In [1]:
from itertools import combinations
from sklearn.linear_model import LogisticRegression

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [4]:
df = pd.read_csv('train.csv')
df.head().values

array([['train_0', 0, 8.9255, ..., 8.5635, 12.7803, -1.0914],
       ['train_1', 0, 11.5006, ..., 8.7889, 18.355999999999998, 1.9518],
       ['train_2', 0, 8.6093, ..., 8.2675, 14.7222, 0.3965],
       ['train_3', 0, 11.0604, ..., 10.2922, 17.9697, -8.9996],
       ['train_4', 0, 9.8369, ..., 9.5031, 17.9974, -8.8104]],
      dtype=object)

In [5]:
# Get some classifiers to evaluate
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC

In [6]:
seed = 1075
np.random.seed(seed)
# Create classifiers
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()

In [7]:
y = df['target'] 
X = df.drop(["ID_code", "target"], axis=1)

In [14]:
names = ['Random Forest', 'Extra Trees', 'KNeighbors', 'SVC', 'Ridge Classifier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [10]:
lr = LogisticRegression()

In [11]:
clf_array = [rf, et, knn, svc, rg]

In [17]:
def zip_stacked_classifiers(*args):
    to_zip = []
    for arg in args:
        combined_items = sum([map(list, combinations(arg, i)) for i in range(len(arg) + 1)], [])
        combined_items = filter(lambda x: len(x) > 0, combined_items)
        to_zip.append(combined_items)
        print('----')   
    return zip(to_zip[0], to_zip[1])
stacked_clf_list = zip_stacked_classifiers(clf_array, names)
# print('----')
best_combination = [0.00, ""]
for clf in stacked_clf_list:
    
    ensemble = SuperLearner(scorer = roc_auc_score, 
                            random_state = seed, 
                            folds = 10)
    ensemble.add(clf[0])
    ensemble.add_meta(lr)
    ensemble.fit(X_train, y_train)
    preds = ensemble.predict(X_test)
    roc_auc_score = roc_auc_score(preds, y_test)
    
    if roc_auc_score > best_combination[0]:
        best_combination[0] = roc_auc_score
        best_combination[1] = clf[1]
    
    print("roc_auc_score: {:.3f} {}").format(roc_auc_score, clf[1])
print("\nBest stacking model is {} with roc_auc_score of: {:.3f}").format(best_combination[1], best_combination[0])

TypeError: can only concatenate list (not "map") to list